# Distilling Knowledge in Multiple Students  using Generative models

In [1]:
                # !nvidia-smi

In [3]:
import numpy as np
import tensorflow as tf
import tensorflow.keras
import tensorflow.keras.backend as K
from tensorflow.keras.datasets import fashion_mnist,mnist,cifar10
from tensorflow.keras.layers import Conv2D,Activation,BatchNormalization,UpSampling2D,Embedding,ZeroPadding2D, Input, Flatten, Dense, Reshape, LeakyReLU, Dropout,MaxPooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras import regularizers

from tensorflow.keras.utils import Progbar
from keras.initializers import RandomNormal
import random
from sklearn.model_selection import train_test_split
from tensorflow.keras import utils as np_utils


In [4]:
# tf.version.VERSION
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

### Teacher's Training

In [5]:
#Loading and splitting the dataset into train, validation and test
nb_classes = 10

(X_Train, y_Train), (X_test, y_test) = mnist.load_data()
X_train, X_val, y_train, y_val = train_test_split(X_Train, y_Train, test_size=0.20)
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_val = np_utils.to_categorical(y_val, nb_classes)

X_train = X_train.reshape(48000, 28, 28, 1)
X_val = X_val.reshape(12000, 28, 28, 1)

X_train = X_train.astype('float32')
X_val = X_val.astype('float32')

# Normalize the values
X_train /= 255
X_val /= 255


#Creating a teacher network
input_shape = (28, 28, 1) # Input shape of each image

teacher = Sequential()
teacher.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
teacher.add(Conv2D(64, (3, 3), activation='relu'))
teacher.add(MaxPooling2D(pool_size=(2, 2)))

teacher.add(Dropout(0.25)) # For reguralization

teacher.add(Flatten())
teacher.add(Dense(256, activation='relu'))
teacher.add(Dense(256, activation='relu', name="dense_1"))

teacher.add(Dropout(0.5)) # For reguralization

teacher.add(Dense(nb_classes, name = 'dense_2'))
teacher.add(Activation('softmax')) # Note that we add a normal softmax layer to begin with

teacher.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(teacher.summary())


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               2359552   
_________________________________________________________________
dense_1 (Dense)              (None, 256)               6

In [4]:
#  Train the teacher model as usual
epochs = 1
batch_size = 256
teacher.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_val, Y_val))


188/188 [==============================] - 5s 15ms/step - loss: 0.6121 - accuracy: 0.8062 - val_loss: 0.0796 - val_accuracy: 0.9764


In [6]:
teacher.load_weights("Teacher_MNIST_98.h5")

### Evaluation 

In [7]:
Y_test = np_utils.to_categorical(y_test, nb_classes)

In [8]:
X_test = X_test.reshape(10000, 28, 28, 1)
X_test = X_test.astype('float32')
X_test /= 255

In [9]:
teacher.evaluate(X_test,Y_test)

313/313 [==============================] - 2s 2ms/step - loss: 0.0541 - accuracy: 0.9869


[0.03862640634179115, 0.9898999929428101]

### Dense Vector split

In [10]:
teacher_WO_Softmax = Model(teacher.input, teacher.get_layer('dense_1').output)
train_dense = teacher_WO_Softmax.predict(X_train)
val_dense = teacher_WO_Softmax.predict(X_val)


# 1 Student case
# ---------------------------------------------
s1Train=train_dense[:,:256]
s1Val=val_dense[:,:256]


### Student

In [21]:
def define_student(name):

    student1 = Sequential()
    student1.add(Conv2D(64, kernel_size=(3, 3),activation='relu',input_shape=(28, 28, 1),kernel_initializer='normal', name=name))
    student1.add(Conv2D(64, (3, 3), activation='relu',kernel_initializer='normal'))
    student1.add(MaxPooling2D(pool_size=(2, 2)))

    student1.add(Conv2D(32, kernel_size=(3, 3),activation='relu',kernel_initializer='normal'))
    student1.add(Conv2D(32, (3, 3), activation='relu',kernel_initializer='normal'))
    student1.add(MaxPooling2D(pool_size=(2, 2)))

    student1.add(Dropout(0.25)) # For reguralization
    student1.add(Flatten())
    student1.add(Dense(16, activation='relu'))
    student1.add(Dropout(0.3))
    student1.add(Dense(256,name='req'+name))

    student1.compile('adam',loss='mean_squared_error',metrics=['accuracy'])
    
    return student1



In [22]:
s1=define_student("s1")
s1.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
s1 (Conv2D)                  (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 24, 24, 64)        36928     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 12, 12, 64)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 10, 10, 32)        18464     
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 8, 8, 32)          9248      
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 4, 4, 32)          0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 4, 4, 32)         

In [23]:
batch_size = 128
s1.fit(X_train, s1Train,
          batch_size=batch_size,
          epochs=20,
          verbose=1, validation_data=(X_val, s1Val))

Epoch 1/20
375/375 [==============================] - 3s 7ms/step - loss: 1.6188 - accuracy: 0.0506 - val_loss: 0.4218 - val_accuracy: 0.3133
Epoch 2/20
375/375 [==============================] - 3s 8ms/step - loss: 0.6821 - accuracy: 0.2276 - val_loss: 0.3657 - val_accuracy: 0.3335
Epoch 3/20
375/375 [==============================] - 3s 8ms/step - loss: 0.6220 - accuracy: 0.2363 - val_loss: 0.3457 - val_accuracy: 0.3571
Epoch 4/20
375/375 [==============================] - 3s 8ms/step - loss: 0.5956 - accuracy: 0.2488 - val_loss: 0.3160 - val_accuracy: 0.3597
Epoch 5/20
375/375 [==============================] - 3s 8ms/step - loss: 0.5674 - accuracy: 0.2587 - val_loss: 0.3219 - val_accuracy: 0.3516
Epoch 6/20
375/375 [==============================] - 3s 8ms/step - loss: 0.5556 - accuracy: 0.2587 - val_loss: 0.3385 - val_accuracy: 0.3489
Epoch 7/20
375/375 [==============================] - 4s 9ms/step - loss: 0.5483 - accuracy: 0.2655 - val_loss: 0.3321 - val_accuracy: 0.3604
Epoch 

### Single Student Configuration

In [24]:
o1=s1.get_layer("reqs1").output

output=Activation('relu')(o1)
output2=Dropout(0.5)(output)
output3=Dense(10,activation="softmax", name="d1")(output2)
mm1=Model(s1.get_layer('s1').input, output3)
my_weights=teacher.get_layer('dense_2').get_weights()

In [25]:
# setting weights
mm1.get_layer('d1').set_weights(my_weights)

In [26]:
# Freezing layers
i=0
for l in mm1.layers[:len(mm1.layers)-1]:
    l.trainable=False
#     print(l)

In [27]:
opt = Adam(lr=0.0002, beta_1=0.5)
mm1.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [28]:
batch_size = 128
mm1.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=20,
          verbose=1, validation_data=(X_val, Y_val))

Epoch 1/20
375/375 [==============================] - 2s 5ms/step - loss: 0.5484 - accuracy: 0.8380 - val_loss: 0.0422 - val_accuracy: 0.9890
Epoch 2/20
375/375 [==============================] - 2s 4ms/step - loss: 0.4471 - accuracy: 0.8508 - val_loss: 0.0493 - val_accuracy: 0.9878
Epoch 3/20
375/375 [==============================] - 2s 6ms/step - loss: 0.4191 - accuracy: 0.8572 - val_loss: 0.0558 - val_accuracy: 0.9877
Epoch 4/20
375/375 [==============================] - 2s 5ms/step - loss: 0.4232 - accuracy: 0.8554 - val_loss: 0.0548 - val_accuracy: 0.9881
Epoch 5/20
375/375 [==============================] - 2s 5ms/step - loss: 0.4196 - accuracy: 0.8566 - val_loss: 0.0559 - val_accuracy: 0.9864
Epoch 6/20
375/375 [==============================] - 2s 4ms/step - loss: 0.4143 - accuracy: 0.8586 - val_loss: 0.0544 - val_accuracy: 0.9866
Epoch 7/20
375/375 [==============================] - 2s 5ms/step - loss: 0.4204 - accuracy: 0.8557 - val_loss: 0.0547 - val_accuracy: 0.9870
Epoch 

In [29]:
loss, acc = mm1.evaluate(X_test, Y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.0542 - accuracy: 0.9890


In [30]:
mm1.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
s1_input (InputLayer)        [(None, 28, 28, 1)]       0         
_________________________________________________________________
s1 (Conv2D)                  (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 24, 24, 64)        36928     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 12, 12, 64)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 10, 10, 32)        18464     
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 8, 8, 32)          9248      
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 4, 4, 32)          0   